In [ ]:
import pandas as pd
from sql_export import sql_connection, sql_connection_close
from azure.storage.blob import BlobServiceClient
import os
from dotenv import load_dotenv
from concurrent import futures


load_dotenv()



## Data Retrieval from Azure SQL

In [ ]:
# Set Apple Silicon paths (for M1/M2 Macs)
# For some reason we need to manually set paths for the odbcinst file so jupyter file can connect to Azure SQL
# Did not need to do this for python code

os.environ['ODBCSYSINI'] = os.getenv('ETC')
os.environ['ODBCINSTINI'] = os.getenv("INI")
os.environ['DYLD_LIBRARY_PATH'] = os.getenv("LIB")


In [ ]:
!odbcinst -q -d -n "ODBC Driver 18 for SQL Server"

In [ ]:
sql_query = "SELECT * FROM dbo.galaxy_target_values"
conn, cursor = sql_connection()
target_df = pd.read_sql(sql_query, conn)
target_df
sql_connection_close()

In [ ]:
target_df

## Data Retrieval From Azure Blob Storage

In [ ]:
connection_string = os.getenv("BLOB_CONNECTION_STRING")


if not connection_string:
    raise Exception("BLOB_CONNECTION_STRING is not defined in your environment variables.")

# connect to Azure blob service
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

containers = blob_service_client.list_containers(include_metadata=False)

blob_container = []
for container in containers:
    blob_container.append(blob_service_client.get_container_client(container=container['name']))

blob_container[0].container_name

if 'test' in blob_container[0].container_name:
    for test_blob in blob_container[0].list_blobs():
        print(test_blob.name)